## Load libraries

In [2]:
import numpy as np
import pandas as pd
import math
import pickle
import json
from gurobipy import *

## Functions

In [3]:
def pickle_load(file_dir):
  with open(file_dir, "rb") as fp:   # Unpickling
    return pickle.load(fp)

def json_load(file_dir):
    with open(file_dir) as f:
        return json.load(f)

In [71]:
data_dir = './data/'

# scenario number & distribution settings
scenario_num = 100
distribution_choice = 'log-normal'

# Sets directory
Bset_dir = data_dir + 'Bset.txt'
Bij_set_dir = data_dir + 'Bij_set.txt'
Sset_dir = data_dir+'Sset_' + str(scenario_num) + '.txt'

deterministic_params_dir = data_dir + 'deterministic_params.json'
capacity_i_dir = data_dir + 'capacity_i.json'
capacity_i_constant_dir = data_dir + 'capacity_i_constant.json'

p_s_dir = data_dir + 'p_s_' + str(scenario_num) + '.json'
demScens_dir = data_dir + 'demScens_' + distribution_choice + '_' + str(scenario_num) + '.dict'

In [104]:
# Sets
Bset = pickle_load(Bset_dir)                                  # 22 Bike Stations
Bij_set = pickle_load(Bij_set_dir)                                  # Bike pairs                                        
Sset = pickle_load(Sset_dir)                                     # Scenario sets
deterministic_params = json_load(deterministic_params_dir)       # deterministic parameters

c = deterministic_params['c']                            # unit procurement cost
v_i = deterministic_params['v_i']                               # stock-out cost
w_i = deterministic_params['w_i']                              # time-waste cost 
t_ij = deterministic_params['t_ij']                    # unit transshipment cost 
capacity_i = json_load(capacity_i_dir)
p_s = json_load(p_s_dir)
demScens = pickle_load(demScens_dir)

The formulation of the two-stage stochastic program problem i written as follows:

<b>First stage varibles:</b> <br>
$$x_i: \text{the number of bikes to assign to bike-station i} \in \text{B at the beginning of the service}$$
<b>Second stage variables: </b>
$$
\beta_{ijs}: \text{Number of rented bikes from bike-station i to bike-station j in scenario s}\\
I_{is}^{+}: \text{Realized surplus of bikes at bike-station i in scenario s} \\
I_{ijs}^{-}: \text{Realized shortage of bikes at origin-destination pair i, j in scenario s}\\
\rho_{ijs}: \text{Number of redirected bikes from bike-station i to bike-station j in scenario s (in case of overflow)}\\
O_{is}^{+}: \text{Residual capacity at bike-station i in scenario s} \\
O_{is}^{-}: \text{Overflow at bike-station i in scenario s} \\
\tau_{ijs}: \text{Number of transshipped bikes from bike-station i to bike-station j in scenario s} \\
T_{is}^{+}: \text{Excess of bikes at bike-station i in scenario s} \\
T_is^{-}: \text{Lack of bikes at bike-station i in scenario s} \\
$$

<b>Formulation:</b>
$$
\begin{align*}
    \text{minimize: }   & c \sum_{i}^B x_i + \sum_{s=1}^S p_s\sum_{i=1}^B [v_i\sum_{j=1}^B I_{ijs}^{-} + w_iO_{is}^{-} + \sum_{j=1}^B t_{ij}\tau_{ijs}]\\
    \text{subject to: }   
        & x_{i} \le k_i, \forall i \in B &\text{(Bike Station Capacity)}\\
        & \beta_{ijs} = \xi_{ijs} - I_{ijs}^{-} , \forall i,j \in B, s \in S &\text{(rented bike successfully made from station i to j)}\\
        & I_{is}^{+} - \sum_{j=1}^B I_{ijs}^{-} = x_i - \sum_{j=1}^B \xi_{ijs} , \forall i \in B, s \in S &\text{(realized surplus and shortage)}\\
        & O_{is}^{+} - O_{is}^{-} = k_i - x_i + \sum_{j=1}^B \beta_{ijs} - \sum_{j=1}^B \beta_{ijs}, \forall i \in B, s \in S &\text{(residual capacity and overflow after rental)}\\
        & \sum_{j=1}^{B} \rho_{ijs} = O_{is}^{-}, \forall i \in B, s \in S &\text{(Redirection identifies station overflow)}\\
        & \sum_{j=1}^{B} \rho_{jis} \le O_{is}^{+}, \forall i \in B, s \in S &\text{(Sucessful redirection less than residual capacity)}\\
        & T_{is}^{+} - T_{is}^{-} = k_i - O_{is}^{+} + \sum_{j=1}^{B}\rho_{jis} - x_i, \forall i \in B, s \in S &\text{(Excess and Lack of bikes at stations)}\\
        & \sum_{j=1}^{B}\tau_{ijs} = T_{is}^{+}, \forall i \in B, s \in S &\text{(Transshipment of excessive bikes)}\\
        & \sum_{j=1}^{B}\tau_{jis} = T_{is}^{-}, \forall i \in B, s \in S &\text{(Transshipment fulfillment)}\\
        & x_i, I_{is}^{+}, O_{is}^{+}, O_{is}^{-}, T_{is}^{+}, T_{is}^{-} \in \mathbb{Z}^{+}, \forall i \in B, s \in S\\
        & \tau_{ijs}, \beta_{ijs}, \rho_{ijs}, I_{ijs}^{-} \in \mathbb{Z}^{+}, , \forall i,j \in B, s \in S\\
\end{align*}
$$

## Building Model

In [105]:
m = Model("2SP_ExtForm")

### Set variables

In [106]:
# first stage variables
x = m.addVars(Bset, vtype=GRB.INTEGER, name='x')

# second stage variables
beta_ijs = m.addVars(Bset,Bset,Sset, vtype=GRB.INTEGER, name='beta_ijs')
I_is_surplus = m.addVars(Bset,Sset, vtype=GRB.INTEGER, name='I_ijs_surplus')
I_ijs_shortage = m.addVars(Bset,Bset, Sset, vtype=GRB.INTEGER, name='I_ijs_shortage')
rho_ijs = m.addVars(Bset,Bset, Sset, vtype=GRB.INTEGER, name='rho_ijs')
O_is_resCap = m.addVars(Bset, Sset, vtype=GRB.INTEGER, name='O_is_resCap')
O_is_overF = m.addVars(Bset, Sset, vtype=GRB.INTEGER, name='O_is_overF')
tau_ijs = m.addVars(Bset,Bset, Sset, vtype=GRB.INTEGER, name='tau_ijs')
T_is_excess = m.addVars(Bset, Sset, vtype=GRB.INTEGER, name='T_is_excess')
T_is_lack = m.addVars(Bset, Sset, vtype=GRB.INTEGER, name='T_is_lack')

m.setObjective(c*quicksum(x[i]for i in Bset) \
               + quicksum(p_s[s]*quicksum(v_i * I_ijs_shortage.sum(i,'*',s)\
                                          + w_i * O_is_overF[(i,s)] \
                                          + t_ij * tau_ijs.sum(i, '*', s) for i in Bset)for s in Sset) )

### Set modelsense

In [107]:
m.modelSense = GRB.MINIMIZE

### Set constraints

In [108]:
m.addConstrs(
    (x[i] <= capacity_i[i] for i in Bset), name='assignment_capacity');

In [109]:
m.addConstrs(
    (beta_ijs[i, j, s] == demScens[i, j, s] - I_ijs_shortage[i, j, s] for i in Bset for j in Bset for s in Sset), \
        name='actual_rental');

In [110]:
m.addConstrs(
    (I_is_surplus[i,s] - I_ijs_shortage.sum(i, '*', s) == x[i] - sum(demScens[i,j,s] for j in Bset) for i in Bset for s in Sset), \
        name='realized_surplus_shortage');

In [111]:
m.addConstrs((O_is_resCap[i,s] - O_is_overF[i,s] \
              == capacity_i[i] - x[i] + beta_ijs.sum(i, '*', s) - beta_ijs.sum('*', i, s) for i in Bset for s in Sset),
            name='residual_overflow_after_rental');

In [112]:
m.addConstrs((rho_ijs.sum(i, '*', s) ==  O_is_overF[i, s] for i in Bset for s in Sset),
            name='redirecting_overflow');

In [113]:
m.addConstrs((rho_ijs.sum('*', i, s) <=  O_is_resCap[i, s] for i in Bset for s in Sset),
            name='successful_redictions');

In [114]:
m.addConstrs((T_is_excess[i, s] - T_is_lack[i, s] \
              == capacity_i[i] - O_is_resCap[i, s] + rho_ijs.sum('*', i, s) - x[i] for i in Bset for s in Sset),
            name='successful_redictions');

In [115]:
m.addConstrs((tau_ijs.sum(i, '*', s) ==  T_is_excess[i, s] for i in Bset for s in Sset),
            name='transshipment_excessive_bike');

In [116]:
m.addConstrs((tau_ijs.sum('*', i, s) ==  T_is_lack[i, s] for i in Bset for s in Sset),
            name='transshipment_fulfillment');

In [117]:
m.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 63822 rows, 204622 columns and 508222 nonzeros
Model fingerprint: 0x172a64b6
Variable types: 0 continuous, 204622 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-04, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+01]
Presolve removed 55022 rows and 86819 columns
Presolve time: 1.00s
Presolved: 8800 rows, 117803 columns, 313743 nonzeros
Variable types: 0 continuous, 117803 integer (18758 binary)

Root relaxation: objective 8.490437e+02, 16410 iterations, 1.16 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     849.0437498  849.04375  0.00%     -    2s

Explored 0 nodes (16410 simplex iterations) in 2.62 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 849.044 

Optimal solu

In [120]:
x

{'A': <gurobi.Var x[A] (value 11.0)>,
 'B': <gurobi.Var x[B] (value 10.0)>,
 'C': <gurobi.Var x[C] (value 10.0)>,
 'D': <gurobi.Var x[D] (value 11.0)>,
 'E': <gurobi.Var x[E] (value 13.0)>,
 'F': <gurobi.Var x[F] (value 12.0)>,
 'G': <gurobi.Var x[G] (value 12.0)>,
 'H': <gurobi.Var x[H] (value 8.0)>,
 'I': <gurobi.Var x[I] (value 8.0)>,
 'J': <gurobi.Var x[J] (value 13.0)>,
 'K': <gurobi.Var x[K] (value 10.0)>,
 'L': <gurobi.Var x[L] (value 13.0)>,
 'M': <gurobi.Var x[M] (value 8.0)>,
 'N': <gurobi.Var x[N] (value 10.0)>,
 'O': <gurobi.Var x[O] (value 10.0)>,
 'P': <gurobi.Var x[P] (value 10.0)>,
 'Q': <gurobi.Var x[Q] (value 13.0)>,
 'R': <gurobi.Var x[R] (value 10.0)>,
 'S': <gurobi.Var x[S] (value 8.0)>,
 'T': <gurobi.Var x[T] (value 12.0)>,
 'U': <gurobi.Var x[U] (value 12.0)>,
 'V': <gurobi.Var x[V] (value 10.0)>}